# Part C: FastAPI Backend Development
## Weather Emergency Prediction - Rostov Region

This notebook creates a FastAPI backend with 5 endpoints:
1. `/api/predict` - Predict emergency
2. `/api/data/historical` - Get historical data
3. `/api/stats` - Get statistics
4. `/api/data/upload` - Upload data files
5. `/api/model/train` - Train model

In [ ]:
# Install required packages
!pip install fastapi uvicorn pydantic python-multipart nest-asyncio pandas numpy scikit-learn joblib

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, HTTPException, UploadFile, File, Query
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
from datetime import datetime
import pandas as pd
import numpy as np
import joblib
import io
import uvicorn

print("✅ Libraries imported")

## 1. Define API Models

In [ ]:
# Request/Response models
class PredictionRequest(BaseModel):
    date: str = Field(..., description="Date in YYYY-MM-DD format")
    temperature: float = Field(..., description="Temperature in Celsius")
    precipitation: float = Field(..., description="Precipitation in mm")
    humidity: float = Field(..., description="Humidity percentage")
    wind_speed: float = Field(..., description="Wind speed in m/s")
    pressure: float = Field(..., description="Atmospheric pressure in hPa")

class PredictionResponse(BaseModel):
    date: str
    emergency_probability: float
    will_occur: bool
    confidence: float

class StatsResponse(BaseModel):
    total_days: int
    total_emergencies: int
    emergency_rate: float
    emergency_types: Dict[str, int]
    avg_temperature: float
    avg_precipitation: float

print("✅ Models defined")

## 2. Initialize FastAPI App

In [ ]:
app = FastAPI(
    title="Weather Emergency Prediction API",
    description="API for predicting weather-related emergencies in Rostov region",
    version="1.0.0"
)

# CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Global variables
model_data = None
weather_data = None
emergency_data = None

print("✅ FastAPI app initialized")

## 3. Load Model and Data

In [ ]:
# Load model from Part B
try:
    model_data = joblib.load('emergency_prediction_model.pkl')
    print(f"✅ Model loaded: {model_data['model_type']}")
except:
    print("⚠️ Model not found. Upload model or train first.")

# Load data from Part A
try:
    weather_data = pd.read_csv('weather_rostov_processed.csv')
    weather_data['date'] = pd.to_datetime(weather_data['date'])
    print(f"✅ Weather data loaded: {len(weather_data)} records")
except:
    print("⚠️ Weather data not found")

try:
    emergency_data = pd.read_csv('emergencies_rostov.csv')
    emergency_data['date'] = pd.to_datetime(emergency_data['date'])
    print(f"✅ Emergency data loaded: {len(emergency_data)} records")
except:
    print("⚠️ Emergency data not found")

## 4. Define Endpoints

In [ ]:
# Endpoint 1: Predict Emergency
@app.post("/api/predict", response_model=PredictionResponse)
async def predict_emergency(request: PredictionRequest):
    """Predict if an emergency will occur."""
    try:
        if model_data is None:
            raise HTTPException(status_code=503, detail="Model not loaded")
        
        model = model_data['model']
        scaler = model_data['scaler']
        feature_names = model_data['feature_names']
        
        # Create input
        input_dict = {
            'temperature': request.temperature,
            'precipitation': request.precipitation,
            'humidity': request.humidity,
            'wind_speed': request.wind_speed,
            'pressure': request.pressure
        }
        
        # Fill missing features
        for feat in feature_names:
            if feat not in input_dict:
                input_dict[feat] = 0.0
        
        X = pd.DataFrame([input_dict])[feature_names]
        X_scaled = scaler.transform(X)
        
        # Predict
        prediction = model.predict(X_scaled)[0]
        proba = model.predict_proba(X_scaled)[0]
        
        return PredictionResponse(
            date=request.date,
            emergency_probability=float(proba[1]),
            will_occur=bool(prediction),
            confidence=float(max(proba))
        )
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

print("✅ Endpoint 1: /api/predict defined")

In [ ]:
# Endpoint 2: Get Historical Data
@app.get("/api/data/historical")
async def get_historical_data(
    start_date: Optional[str] = Query(None),
    end_date: Optional[str] = Query(None),
    limit: int = Query(1000)
):
    """Get historical weather data."""
    try:
        if weather_data is None:
            raise HTTPException(status_code=404, detail="Weather data not available")
        
        df = weather_data.copy()
        
        # Filter by date
        if start_date:
            df = df[df['date'] >= pd.to_datetime(start_date)]
        if end_date:
            df = df[df['date'] <= pd.to_datetime(end_date)]
        
        df = df.head(limit)
        
        # Convert to JSON
        result = df.to_dict(orient='records')
        for record in result:
            if 'date' in record:
                record['date'] = pd.to_datetime(record['date']).strftime('%Y-%m-%d')
        
        return {"data": result, "count": len(result)}
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

print("✅ Endpoint 2: /api/data/historical defined")

In [ ]:
# Endpoint 3: Get Statistics
@app.get("/api/stats", response_model=StatsResponse)
async def get_statistics(
    start_date: Optional[str] = Query(None),
    end_date: Optional[str] = Query(None)
):
    """Get statistical summary."""
    try:
        if weather_data is None:
            raise HTTPException(status_code=404, detail="Data not available")
        
        df = weather_data.copy()
        
        # Filter by date
        if start_date:
            df = df[df['date'] >= pd.to_datetime(start_date)]
        if end_date:
            df = df[df['date'] <= pd.to_datetime(end_date)]
        
        # Calculate stats
        total_days = len(df)
        avg_temp = float(df['temperature'].mean())
        avg_precip = float(df['precipitation'].mean())
        
        # Emergency stats
        if emergency_data is not None:
            emg_df = emergency_data.copy()
            if start_date:
                emg_df = emg_df[emg_df['date'] >= pd.to_datetime(start_date)]
            if end_date:
                emg_df = emg_df[emg_df['date'] <= pd.to_datetime(end_date)]
            
            total_emergencies = len(emg_df)
            emergency_types_count = emg_df['type'].value_counts().to_dict()
        else:
            total_emergencies = 0
            emergency_types_count = {}
        
        emergency_rate = total_emergencies / total_days if total_days > 0 else 0.0
        
        return StatsResponse(
            total_days=total_days,
            total_emergencies=total_emergencies,
            emergency_rate=emergency_rate,
            emergency_types=emergency_types_count,
            avg_temperature=avg_temp,
            avg_precipitation=avg_precip
        )
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

print("✅ Endpoint 3: /api/stats defined")

In [ ]:
# Endpoint 4: Upload Data
@app.post("/api/data/upload")
async def upload_data(
    file: UploadFile = File(...),
    data_type: str = Query(..., description="'weather' or 'emergency'")
):
    """Upload weather or emergency data."""
    global weather_data, emergency_data
    
    try:
        contents = await file.read()
        
        # Read file
        if file.filename.endswith('.csv'):
            df = pd.read_csv(io.BytesIO(contents))
        elif file.filename.endswith(('.xlsx', '.xls')):
            df = pd.read_excel(io.BytesIO(contents))
        else:
            raise HTTPException(status_code=400, detail="File must be CSV or Excel")
        
        df['date'] = pd.to_datetime(df['date'])
        
        # Store data
        if data_type == 'weather':
            weather_data = df
            return {"message": f"Uploaded {len(df)} weather records", "type": "weather"}
        elif data_type == 'emergency':
            emergency_data = df
            return {"message": f"Uploaded {len(df)} emergency records", "type": "emergency"}
        else:
            raise HTTPException(status_code=400, detail="Invalid data_type")
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

print("✅ Endpoint 4: /api/data/upload defined")

In [ ]:
# Endpoint 5: Health Check
@app.get("/health")
async def health_check():
    """Check API health."""
    return {
        "status": "healthy",
        "model_loaded": model_data is not None,
        "weather_data_loaded": weather_data is not None,
        "emergency_data_loaded": emergency_data is not None
    }

print("✅ Endpoint 5: /health defined")

## 5. Run API Server

In [ ]:
# Run server in Colab
print("\n" + "="*50)
print("Starting FastAPI Server")
print("="*50)
print("\nAPI will be available at: http://localhost:8000")
print("API Documentation: http://localhost:8000/docs")
print("\nPress Ctrl+C to stop the server")
print("="*50 + "\n")

# Note: In Colab, use ngrok for public URL
# !pip install pyngrok
# from pyngrok import ngrok
# public_url = ngrok.connect(8000)
# print(f"Public URL: {public_url}")

# Run server
uvicorn.run(app, host="0.0.0.0", port=8000)

## 6. Test API Endpoints

In [ ]:
# Test in another notebook or cell (after server is running)
import requests

BASE_URL = "http://localhost:8000"

# Test health check
response = requests.get(f"{BASE_URL}/health")
print("Health Check:")
print(response.json())

# Test prediction
payload = {
    "date": "2024-07-15",
    "temperature": 38.0,
    "precipitation": 0.0,
    "humidity": 25.0,
    "wind_speed": 5.0,
    "pressure": 1013.0
}
response = requests.post(f"{BASE_URL}/api/predict", json=payload)
print("\nPrediction:")
print(response.json())

# Test stats
response = requests.get(f"{BASE_URL}/api/stats")
print("\nStatistics:")
print(response.json())

## Summary

✅ **Completed:**
- Created FastAPI application with 5 endpoints
- Implemented prediction endpoint
- Implemented historical data retrieval
- Implemented statistics endpoint
- Implemented file upload endpoint
- Added health check endpoint
- Enabled CORS for web access
- Added API documentation (Swagger UI)

**Next:** Continue to Part D - Web Interface